In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from standard_libs import *

<a id='ch06'></a>
# Chapter 6 - Learning to Classify Text
* [Section 6.1 - Supervised Classification](#section1)
    * [Section 6.1.3 - Classify Documents](#section13)
    * [Section 6.1.5 - Exploiting Context](#section15)
    * [Section 6.1.6 - Sequence Classification](#section16)
    * [Section 6.1.7 - Other classification methods](#section17)

<a id='section1'></a>
## 6.1 Supervised Classification
### Gender Identification
Use the ending character to predict whether a name is male or female

In [2]:
def gender_features(word):
    return {'last_letter': word[-1]}

In [3]:
from nltk.corpus import names

In [4]:
labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
                [(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

Use the feature extractor to process the `names` data, and divide the resulting list of feature sets into a **training set** and **test set**.

In [5]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [7]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     34.4 : 1.0
             last_letter = 'k'              male : female =     33.0 : 1.0
             last_letter = 'f'              male : female =     16.1 : 1.0
             last_letter = 'p'              male : female =     10.6 : 1.0
             last_letter = 'v'              male : female =      9.9 : 1.0


#### Exercise: Add first letter and name length - doesn't seem to make much difference in prediction accuracy

In [8]:
def gender_features(word):
    return {'last_letter': word[-1], 'first_letter': word[0], 'len': len(word)}

In [9]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [10]:
print(nltk.classify.accuracy(classifier, test_set))

0.764


To avoid using too much memory by storing as a single list the features of every instance, use `nltk.classify.apply_features`

In [14]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

### How to improve model using a dev set

In [15]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [16]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
dev_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, dev_set))

0.786


Using the dev set, we can generate a list of the errors that the classifiers makes when predicting name genders:

In [17]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

In [ ]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

In [25]:
def gender_features2(word):
    return {'suffix1': word[-1:], 'suffix2': word[-2:], 'len': len(word), 'first': word[0]}

In [26]:
train_set = [(gender_features2(n), gender) for (n, gender) in train_names]
dev_set = [(gender_features2(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features2(n), gender) for (n, gender) in test_names]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, dev_set))

0.786


<a id="section13"></a>
### 1.3 Document Classification
[Back](#ch06)

Movie reviews

In [29]:
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [30]:
# extract 2000 most frequent words in the overall corpus - define feature extractor that simply checks whether each of these words is present in a given document
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

In [33]:
featuresets = [(document_features(d), c) for (d, c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [34]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

0.75
Most Informative Features
 contains(unimaginative) = True              neg : pos    =      7.8 : 1.0
    contains(schumacher) = True              neg : pos    =      7.5 : 1.0
     contains(atrocious) = True              neg : pos    =      6.7 : 1.0
          contains(mena) = True              neg : pos    =      6.4 : 1.0
        contains(shoddy) = True              neg : pos    =      6.4 : 1.0


<a id='section14'></a>
### 1.4 Part of Speech Tagging
[Back](#ch06)

We can train a classifier to work out which suffixes are most informative.

In [35]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [37]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]

In [38]:
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


Define a feature extractor function which checks a given word for these suffixes:

In [39]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [40]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n, g) in tagged_words]

In [41]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [42]:
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [43]:
classifier.classify(pos_features('cats'))

'NNS'

In [44]:
print(classifier.pseudocode(depth=4))

if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



<a id="section15"></a>
### 1.5 Exploiting Context
[Back](#ch06)

By augmenting the feature extraction function, we could modify this part-of-speech tagger to leverage a variety of other word-internal features,such as the length of the word. However, as long as the feature extractor just looks at the target word, we have no way to add features that depend on the *context* that the word appears in. But contextual features often provide powerful clues about the correct tag.

In the new feature extraction method, we will pass in a complete (untagged) sentence, along with the index of the target word. This employs a context-dependent feature extractor to define a part of speech tag classifier.

In [46]:
def pos_features(sentence, i):
    features = {'suffix(1)': sentence[i][-1:],
                'suffix(2)': sentence[i][-2:],
                'suffix(3)': sentence[i][-3:]}
    if i == 0:
        features['prev-word'] = '<START>'
    else:
        features['prev-word'] = sentence[i-1]
    return features

In [47]:
pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [48]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag))

In [49]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

<a id='section16'></a>
### 1.6 Sequence Classification
[Back](#ch06)

To capture the dependencies between related classification tasks, we can use **joint classifier** models, which choose an appropriate labeling for a collection of related inputs. In the case of POS tagging, a variety of different **sequence classifier** models can be used to jointly choose part-of-speech tags for all the words in a given sentence.

#### Consecutive classification or greedy sequence classification
Based on history.

In [50]:
def pos_features(sentence, i, history):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:], 
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features['prev-word'] = "<START>"
        features['prev-tag'] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features

In [51]:
class ConsecutivePosTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent, i, history)
                train_set.append((featureset, tag))
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
        
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [52]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

0.7980528511821975


<a id="section17"></a>
### 1.7 Other Methods for Sequence Classification
[Back](#ch06)

One shortcoming of this approach is that we commit to every decision that we make. For example, if we decide to label a word as a noun, but later find evidence that it should have been a verb, there's no way to go back and fix our mistake. One solution is to adopt a transformational strategy instead. Transformational joint classifiers work by creating an initial assignment of labels for the inputs, and then iteratively refining that assignment in an attempt to repair inconsistencies between related inputs.

Another solution is to assign scores to all of the possible sequences of part-of-speech tags, and to choose the sequence whose overall score is highest. This is the approach taken by **Hidden Markov Models**. Hidden Markov Models are similar to consecutive classifiers in that they look at both the inputs and the history of predicted tags. However, rather than simply finding the single best tag for a given word, they generate a probability distribution over tags. These probabilities are then combined to calculate probability scores for tag sequences, and the tag sequence with the highest probability is chosen.